# Estrategia Pair-Trading

La idea detras de esta estrategia de trading es que los stocks relacionados, que tienen cierta correlacion, tienden a moverse juntos.

Por ejemplo, si se encuentra un comportamiento inusual, donde se separan mucho ambos stocks (suponiendo correlacion positiva), se entra en corto a la mas alta y en largo a la mas baja de las dos. A la separacion existente entre ambos stocks se le llama 'spread', cuanto mas largo sea este, mas largo podra ser el trade.

Esta estrategia es muy simple, pero sirve al proposito de crear una aplicacion web con Python desde cero..se ira mejorando.

# Datos

Se usaran datos del Dow Jones Industrial, de las 10 compañias que mas peso tienen.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
index_symbol = ['^DJI']  # Dow Jones Industrial


stock_symbols = ['BA',    # Boeing
                 'GS',    # Goldman Sachs
                 'UNH',   # UnitedHealth Group
                 'MMM',   # 3M
                 'HD',    # Home Depot
                 'AAPL',  # Apple
                 'MCD',   # McDonalds
                 'IBM',   # IBM
                 'CAT',   # Caterpillar
                 'TRV']   # Travelers

**para datos en directo o actualizados [yahoo finance](https://finance.yahoo.com)**